In [44]:
# python -m streamlit run prevert.py

from collections import defaultdict
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
# from utils import tokenize

from google.oauth2 import service_account
import streamlit as st
credentials = service_account.Credentials.from_service_account_info(st.secrets["gcp_service_account"])

def tokenize(text):
    return re.sub(' +', ' ', re.sub("\W+", ' ', unidecode(text.lower()))).strip(' \n')



In [45]:
with open('bq_view_updated.sql', 'r') as f:
    query_view = f.read()
df_updated = pd.read_gbq(query_view, credentials=credentials)
df_updated.head()

,timestamp,updated_timestamp,quote,extra,info,is_delete
0,2024-11-19 22:10:51.069501+00:00,2024-11-19 22:10:51.069501+00:00,"{'title': 'None', 'author': 'Scooby-Doo', 'tex...","{'quote_react': '𝄞', 'note': None}","{'text_tok': 'Scoobydoobydoo-music !!!', 'nb_c...",False
1,2024-11-19 23:52:05.722450+00:00,2024-11-19 23:52:05.722450+00:00,"{'title': 'None', 'author': 'Scooby-Doo', 'tex...","{'quote_react': '𝄞', 'note': None}","{'text_tok': 'Scoobydoobydoo Fire !', 'nb_char...",False
2,2024-11-19 23:52:05.722450+00:00,2024-11-19 23:52:05.722450+00:00,"{'title': 'None', 'author': 'Scooby-Doo', 'tex...","{'quote_react': '𝄞', 'note': None}","{'text_tok': 'Scoobydoobydoo Fire !', 'nb_char...",False
3,2024-11-18 00:16:09.377227+00:00,2024-11-18 00:16:09.377227+00:00,"{'title': 'None', 'author': 'tiktok', 'text': ...","{'quote_react': '𝄞', 'note': None}",{'text_tok': 'Un oiseau ne chante pas parce qu...,False
4,2024-11-17 23:42:45.546705+00:00,2024-11-17 23:42:45.546705+00:00,"{'title': 'None', 'author': 'Thomas', 'text': ...","{'quote_react': '𝄞', 'note': None}",{'text_tok': 'La danse m'a appris davantage qu...,False


In [58]:
df_updated.iloc[2]

text                                Scoobydoobydoo Fire !
author                                         Scooby-Doo
book                                                 None
title                                                None
nb_like                                              2700
haiku                                               False
quote_react                                             𝄞
note                                                 None
all_search     scoobydoobydoo fire scooby doo none none 𝄞
is_delete                                           False
Name: Scoobydoobydoo Fire !, dtype: object

In [46]:
df_updated['info'].apply(lambda x: x['text_tok'])
df_updated.iloc[23:26]

,timestamp,updated_timestamp,quote,extra,info,is_delete
23,2024-11-17 00:00:00+00:00,2024-11-18 23:45:42.184566+00:00,"{'title': None, 'author': 'Pierre Desproges', ...","{'quote_react': None, 'note': None}",{'text_tok': 'les enfants c est comme les pets...,True
24,2024-11-17 00:00:00+00:00,2024-11-18 23:45:42.184566+00:00,"{'title': None, 'author': 'Yasmina Khadra', 't...","{'quote_react': None, 'note': None}",{'text_tok': 'celui qui passe a cote de la plu...,<NA>
25,2024-11-17 00:00:00+00:00,2024-11-19 23:47:20.070699+00:00,"{'title': None, 'author': 'Kerouac', 'text': '...","{'quote_react': '🌚🌚🌚🌚🌚🌚🎶', 'note': 'Useless ! ...",{'text_tok': 'inutile inutile la pluie lourde ...,<NA>


In [47]:
df_updated['text'] = df_updated['quote'].apply(lambda x: x.get('text', ''))
df_updated['text_tok'] = df_updated['quote'].apply(lambda x: tokenize(x.get('text', '')))
df_updated['author'] = df_updated['quote'].apply(lambda x: x.get('author', ''))
df_updated['book'] = df_updated['quote'].apply(lambda x: x.get('book', ''))
df_updated['title'] = df_updated['quote'].apply(lambda x: x.get('title', ''))
df_updated['quote_react'] = df_updated['extra'].apply(lambda x: x.get('quote_react', ''))
df_updated['note'] = df_updated['extra'].apply(lambda x: x.get('note', ''))
df_updated['haiku'] = df_updated['text'].apply(lambda x: len(x.split('\n')) == 3)
df_updated['nb_like'] = 0
df_updated['all_search'] = (df_updated['text_tok'].astype(str)) + (df_updated['author'].astype(str).apply(tokenize)) + (df_updated['book'].astype(str).apply(tokenize)) + (df_updated['title'].astype(str).apply(tokenize)) + df_updated['quote_react'].astype(str)

df_updated = df_updated[['text', 'author', 'book', 'title', 'nb_like', 'text_tok', 'haiku', 'quote_react', 'note', 'all_search', 'is_delete']]
# df_updated.set_index('text_tok', inplace = True, drop = True)
df_updated.iloc[23:26]

,text,author,book,title,nb_like,text_tok,haiku,quote_react,note,all_search,is_delete
23,"Les enfants, c'est comme les pets. Il n'y a qu...",Pierre Desproges,None,None,0,les enfants c est comme les pets il n y a que ...,False,None,None,les enfants c est comme les pets il n y a que ...,True
24,Celui qui passe à côté de la plus belle histoi...,Yasmina Khadra,Ce que le jour doit à la nuit,None,0,celui qui passe a cote de la plus belle histoi...,False,None,None,celui qui passe a cote de la plus belle histoi...,<NA>
25,Inutile ! inutile ! \n– la pluie lourde s’enf...,Kerouac,Livre des haïkus,None,0,inutile inutile la pluie lourde s enfonce dans...,True,🌚🌚🌚🌚🌚🌚🎶,Useless ! Useless ! \n— heavy rain driving \...,inutile inutile la pluie lourde s enfonce dans...,<NA>


In [48]:
df_updated = df_updated[['text', 'author', 'book', 'title', 'nb_like', 'text_tok', 'haiku', 'quote_react', 'note', 'all_search', 'is_delete']]
df_updated.set_index('text_tok', inplace = True, drop = True)
df_updated.iloc[23:26]

,text,author,book,title,nb_like,haiku,quote_react,note,all_search,is_delete
text_tok,,,,,,,,,,
les enfants c est comme les pets il n y a que les siens qu on peut supporter,"Les enfants, c'est comme les pets. Il n'y a qu...",Pierre Desproges,None,None,0,False,None,None,les enfants c est comme les pets il n y a que ...,True
celui qui passe a cote de la plus belle histoire de sa vie n aura que l age de ses regrets et tous les soupirs du monde ne sauraient bercer son ame,Celui qui passe à côté de la plus belle histoi...,Yasmina Khadra,Ce que le jour doit à la nuit,None,0,False,None,None,celui qui passe a cote de la plus belle histoi...,<NA>
inutile inutile la pluie lourde s enfonce dans la mer,Inutile ! inutile ! \n– la pluie lourde s’enf...,Kerouac,Livre des haïkus,None,0,True,🌚🌚🌚🌚🌚🌚🎶,Useless ! Useless ! \n— heavy rain driving \...,inutile inutile la pluie lourde s enfonce dans...,<NA>


In [ ]:
df_updated[df_updated['author'] == 'Scooby-Doo']

,text,author,book,title,nb_like,haiku,quote_react,note,all_search,is_delete
text_tok,,,,,,,,,,
Scoobydoobydoo-music !!!,Scoobydoobydoo-music !!!,Scooby-Doo,None,None,2700,False,𝄞,None,scoobydoobydoo music scooby doo none none 𝄞,False
Scoobydoobydoo Fire !,Scoobydoobydoo Fire !,Scooby-Doo,None,None,2700,False,𝄞,None,scoobydoobydoo fire scooby doo none none 𝄞,False
Scoobydoobydoo Fire !,Scoobydoobydoo Fire !,Scooby-Doo,None,None,2700,False,𝄞,None,scoobydoobydoo fire scooby doo none none 𝄞,False
Scoobydoobydoo !,Scoobydoobydoo !,Scooby-Doo,None,None,2700,False,𝄞,None,scoobydoobydoo scooby doo none none 𝄞,False
Scoobydoobydoo !,Scoobydoobydoo !,Scooby-Doo,None,None,2700,False,𝄞,None,scoobydoobydoo scooby doo none none 𝄞,False
Scoobydoobydoo !,Scoobydoobydoo !,Scooby-Doo,None,None,2700,False,𝄞,None,scoobydoobydoo scooby doo none none 𝄞,False
Scoobydoobydoo !,Scoobydoobydoo !,Scooby-Doo,None,None,2700,False,𝄞,None,scoobydoobydoo scooby doo none none 𝄞,False
Scoobydoobydoo !,Scoobydoobydoo !,Scooby-Doo,None,None,2700,False,𝄞,None,scoobydoobydoo scooby doo none none 𝄞,False
Scooby-Doo,Scooby-Doo,Scooby-Doo,None,Scooby-Doo,2700,False,𝄞,None,scooby doo scooby doo none scooby doo 𝄞,False


In [49]:
df_updated.shape

(54, 10)

In [50]:
# data_ram = pd.read_parquet('data_v2/data_ram.parquet')
# data_ram[data_ram['author'] == "Scooby-Doo"]
data_v1 = pd.read_parquet('data_v2/raw_data_v1_17_nov_.parquet')

In [51]:
data_v1['note'] = data_v1['vo']
data_v1['all_search'] = (data_v1['text_tok'].astype(str)) + (data_v1['author'].astype(str).apply(tokenize)) + (data_v1['book'].astype(str).apply(tokenize)) + (data_v1['title'].astype(str).apply(tokenize)) + data_v1['quote_react'].astype(str)
data_v1.set_index('text_tok', inplace = True, drop = True)
data_v1.drop(['year', 'source', 'confiance', 'page', 'url', 'nb_char', 'nb_lines', 'sonnet', 'vo'], axis = 1, inplace = True)
data_v1['is_delete'] = False
print('data_v1 loaded')

# add events
with open('bq_view_updated.sql', 'r') as f:
    query_view = f.read()
df_updated = pd.read_gbq(query_view, credentials=credentials)
print(f'{len(df_updated)} events loaded')

df_updated['text'] = df_updated['quote'].apply(lambda x: x.get('text', ''))
df_updated['text_tok'] = df_updated['quote'].apply(lambda x: x.get('text', '')) # no tokenize (events text_tok are consider as is (will be tokenize in all_search))
df_updated['author'] = df_updated['quote'].apply(lambda x: x.get('author', ''))
df_updated['book'] = df_updated['quote'].apply(lambda x: x.get('book', ''))
df_updated['title'] = df_updated['quote'].apply(lambda x: x.get('title', ''))
df_updated['quote_react'] = df_updated['extra'].apply(lambda x: x.get('quote_react', ''))
df_updated['note'] = df_updated['extra'].apply(lambda x: x.get('note', ''))
df_updated['haiku'] = df_updated['text'].apply(lambda x: len(x.split('\n')) == 3)
df_updated['nb_like'] = len(df_updated['quote_react']) * 50
df_updated['all_search'] = (df_updated['text_tok'].astype(str).apply(tokenize)) + ' ' + (df_updated['author'].astype(str).apply(tokenize)) +' ' + (df_updated['book'].astype(str).apply(tokenize)) + ' ' +(df_updated['title'].astype(str).apply(tokenize)) +' ' + df_updated['quote_react'].astype(str)

df_updated = df_updated[['text', 'author', 'book', 'title', 'nb_like', 'text_tok', 'haiku', 'quote_react', 'note', 'all_search', 'is_delete']]
df_updated.set_index('text_tok', inplace = True, drop = True)

data_ram = pd.concat([df_updated, data_v1], axis = 0)
data_ram['is_delete'] = data_ram['is_delete'].replace(np.nan, False)
# drop duplicates index
data_ram = data_ram[~data_ram.index.duplicated(keep='first')]
data_ram.sort_values('nb_like', ascending = False, inplace = True)
data_ram = data_ram[~data_ram['is_delete']]
data_ram.drop(['is_delete', 'all_search'], axis = 1, inplace = True)
# data_ram.drop(['is_delete'], axis = 1, inplace = True)

data_ram['author'] = data_ram['author'].astype(str)
data_ram['book'] = data_ram['book'].astype(str).replace('None', None)
data_ram['title'] = data_ram['title'].astype(str).replace('nan', None)
data_ram['quote_react'] = data_ram['quote_react'].astype(str).replace('None', None)
data_ram['note'] = data_ram['note'].astype(str).replace('nan', None)
# data_ram['all_search'] = data_ram['all_search'].astype(str)
data_ram['text'] = data_ram['text'].astype(str)
data_ram['haiku'] = data_ram['haiku'].astype(bool)
data_ram['nb_like'] = data_ram['nb_like'].astype(int)

data_v1 loaded
54 events loaded


In [54]:
data_ram[data_ram['author'] == 'Scooby-Doo']

,text,author,book,title,nb_like,haiku,quote_react,note
text_tok,,,,,,,,
Scoobydoobydoo-music !!!,Scoobydoobydoo-music !!!,Scooby-Doo,None,None,2700,False,𝄞,None
Scoobydoobydoo Fire !,Scoobydoobydoo Fire !,Scooby-Doo,None,None,2700,False,𝄞,None
Scoobydoobydoo !,Scoobydoobydoo !,Scooby-Doo,None,None,2700,False,𝄞,None
Scooby-Doo,Scooby-Doo,Scooby-Doo,None,Scooby-Doo,2700,False,𝄞,None


In [35]:
def get_kerouac(data_ram):
    z = data_ram[data_ram['author'] == "Kerouac"]
    pattern = "la pluie lourde"
    return z[z['text'].apply(lambda x: pattern in x).values]

get_kerouac(data_ram)

,text,author,book,title,nb_like,haiku,quote_react,note
text_tok,,,,,,,,
Inutile ! inutile ! \n– la pluie lourde s’enfonce \nDans la mer,Inutile ! inutile ! \n– la pluie lourde s’enf...,Kerouac,Livre des haïkus,None,2600,True,🌚🌚🌚🌚🎶,Useless ! Useless ! \n— heavy rain driving \...
\nInutile ! Inutile ! \n– la pluie lourde s’enfonce \nS’enfonce dans la mer,\nInutile ! Inutile ! \n– la pluie lourde...,Kerouac,Livre des haïkus,None,2600,False,🌚🎶,Useless ! Useless ! \n— heavy rain driving \...
inutile inutile la pluie lourde s enfonce dans la mer,Inutile ! inutile ! \n– la pluie lourde s’enf...,Kerouac,Livre des haïkus,None,150,True,🎶,Useless ! Useless ! \n— heavy rain driving \...
inutile inutile la pluie lourde s enfonce s enfonce dans la mer13,\nInutile ! Inutile ! \n– la pluie lourde...,Kerouac,Livre des haïkus,None,50,False,🎶,Useless ! Useless ! \n— heavy rain driving \...
inutile inutile la pluie lourde s enfonce s enfonce dans la mer,\nInutile ! Inutile ! \n– la pluie lourde...,Kerouac,Livre des haïkus,None,50,False,None,Useless ! Useless ! \n— heavy rain driving \...


In [30]:
data_ram.shape

(56297, 8)

In [34]:
def load_data():
    data_ram = pd.read_parquet('data_v2/data_ram.parquet')
    # data_ram.set_index('text_tok', inplace = True, drop = True)
    data_ram['all_search'] = (data_ram.index.astype(str)) + (data_ram['author'].astype(str).apply(tokenize))+ (data_ram['text'].astype(str).apply(tokenize)) + (data_ram['book'].astype(str).apply(tokenize)) + (data_ram['title'].astype(str).apply(tokenize)) + data_ram['quote_react'].astype(str)
    data_ram.sort_values('nb_like', ascending = False, inplace = True)
    return data_ram
dr = load_data()
get_kerouac(dr)

,text,author,book,title,nb_like,haiku,quote_react,note,all_search
text_tok,,,,,,,,,
Inutile ! inutile ! \n– la pluie lourde s’enfonce \nDans la mer,Inutile ! inutile ! \n– la pluie lourde s’enf...,Kerouac,Livre des haïkus,None,2600,True,🌚🌚🌚🌚🎶,Useless ! Useless ! \n— heavy rain driving \...,Inutile ! inutile ! \n– la pluie lourde s’enf...
inutile inutile la pluie lourde s enfonce dans la mer,Inutile ! inutile ! \n– la pluie lourde s’enf...,Kerouac,Livre des haïkus,None,150,True,🎶,Useless ! Useless ! \n— heavy rain driving \...,inutile inutile la pluie lourde s enfonce dans...


In [ ]:
# create events_encoded
with open('bq_view_updated.sql', 'r') as f:
    query_view = f.read()
events = pd.read_gbq(query_view, credentials=credentials)
events['text_tok'] = events['info'].apply(lambda x: x['text_tok']).apply(tokenize)
events
# (CURRENT_timestamp(), "Inutile ! inutile !  \n– la pluie lourde s’enfonce  \nDans la mer", "localhost", "delete", "None", "None", "None", "None", "None") ,
# 

,timestamp,updated_timestamp,quote,extra,info,is_delete,text_tok
0,2024-11-18 00:16:09.377227+00:00,2024-11-18 00:16:09.377227+00:00,"{'title': 'None', 'author': 'tiktok', 'text': ...","{'quote_react': '𝄞', 'note': None}",{'text_tok': 'Un oiseau ne chante pas parce qu...,False,un oiseau ne chante pas parce qu il a une repo...
1,2024-11-17 23:42:45.546705+00:00,2024-11-17 23:42:45.546705+00:00,"{'title': 'None', 'author': 'Thomas', 'text': ...","{'quote_react': '𝄞', 'note': None}",{'text_tok': 'La danse m'a appris davantage qu...,False,la danse m a appris davantage que de deplacer ...
2,2024-11-17 23:41:36.402954+00:00,2024-11-17 23:41:36.402954+00:00,"{'title': 'None', 'author': 'Thomas', 'text': ...","{'quote_react': '𝄞', 'note': None}",{'text_tok': 'La danse m'a appris davantage qu...,False,la danse m a appris davantage que de deplacer ...
3,2024-11-17 23:42:45.546705+00:00,2024-11-17 23:42:45.546705+00:00,"{'title': 'None', 'author': 'tiktok', 'text': ...","{'quote_react': '𝄞', 'note': None}",{'text_tok': 'Un oiseau perché sur un arbre n'...,False,un oiseau perche sur un arbre n a jamais peur ...
4,2024-11-17 23:42:45.546705+00:00,2024-11-17 23:42:45.546705+00:00,"{'title': 'None', 'author': 'tiktok', 'text': ...","{'quote_react': '𝄞', 'note': None}","{'text_tok': 'Au départ, le peintre a une toil...",False,au depart le peintre a une toile l ecrivain a ...
5,2024-11-17 23:41:36.402954+00:00,2024-11-17 23:41:36.402954+00:00,"{'title': 'None', 'author': 'tiktok', 'text': ...","{'quote_react': '𝄞', 'note': None}","{'text_tok': 'Au départ, le peintre a une toil...",False,au depart le peintre a une toile l ecrivain a ...
6,2024-11-17 23:41:36.402954+00:00,2024-11-17 23:41:36.402954+00:00,"{'title': 'None', 'author': 'tiktok', 'text': ...","{'quote_react': '𝄞', 'note': None}",{'text_tok': 'Un oiseau perché sur un arbre n'...,False,un oiseau perche sur un arbre n a jamais peur ...
7,2024-11-18 00:12:44.219139+00:00,2024-11-18 00:12:44.219139+00:00,"{'title': 'None', 'author': 'tiktok', 'text': ...","{'quote_react': '𝄞', 'note': None}",{'text_tok': 'Un oiseau ne chante pas parce qu...,False,un oiseau ne chante pas parce qu il a une repo...
8,2024-11-19 22:06:24.169798+00:00,2024-11-19 22:06:24.169798+00:00,"{'title': 'None', 'author': 'Scooby-Doo', 'tex...","{'quote_react': '𝄞', 'note': None}","{'text_tok': 'Scoobydoobydoo !', 'nb_char': No...",False,scoobydoobydoo
9,2024-11-19 22:06:24.169798+00:00,2024-11-19 22:06:24.169798+00:00,"{'title': 'None', 'author': 'Scooby-Doo', 'tex...","{'quote_react': '𝄞', 'note': None}","{'text_tok': 'Scoobydoobydoo !', 'nb_char': No...",False,scoobydoobydoo


In [42]:
events

,timestamp,updated_timestamp,quote,extra,info,is_delete
0,2024-11-18 00:16:09.377227+00:00,2024-11-18 00:16:09.377227+00:00,"{'title': 'None', 'author': 'tiktok', 'text': ...","{'quote_react': '𝄞', 'note': None}",{'text_tok': 'Un oiseau ne chante pas parce qu...,False
1,2024-11-17 23:42:45.546705+00:00,2024-11-17 23:42:45.546705+00:00,"{'title': 'None', 'author': 'Thomas', 'text': ...","{'quote_react': '𝄞', 'note': None}",{'text_tok': 'La danse m'a appris davantage qu...,False
2,2024-11-17 23:41:36.402954+00:00,2024-11-17 23:41:36.402954+00:00,"{'title': 'None', 'author': 'Thomas', 'text': ...","{'quote_react': '𝄞', 'note': None}",{'text_tok': 'La danse m'a appris davantage qu...,False
3,2024-11-17 23:42:45.546705+00:00,2024-11-17 23:42:45.546705+00:00,"{'title': 'None', 'author': 'tiktok', 'text': ...","{'quote_react': '𝄞', 'note': None}",{'text_tok': 'Un oiseau perché sur un arbre n'...,False
4,2024-11-17 23:42:45.546705+00:00,2024-11-17 23:42:45.546705+00:00,"{'title': 'None', 'author': 'tiktok', 'text': ...","{'quote_react': '𝄞', 'note': None}","{'text_tok': 'Au départ, le peintre a une toil...",False
5,2024-11-17 23:41:36.402954+00:00,2024-11-17 23:41:36.402954+00:00,"{'title': 'None', 'author': 'tiktok', 'text': ...","{'quote_react': '𝄞', 'note': None}","{'text_tok': 'Au départ, le peintre a une toil...",False
6,2024-11-17 23:41:36.402954+00:00,2024-11-17 23:41:36.402954+00:00,"{'title': 'None', 'author': 'tiktok', 'text': ...","{'quote_react': '𝄞', 'note': None}",{'text_tok': 'Un oiseau perché sur un arbre n'...,False
7,2024-11-18 00:12:44.219139+00:00,2024-11-18 00:12:44.219139+00:00,"{'title': 'None', 'author': 'tiktok', 'text': ...","{'quote_react': '𝄞', 'note': None}",{'text_tok': 'Un oiseau ne chante pas parce qu...,False
8,2024-11-19 22:06:24.169798+00:00,2024-11-19 22:06:24.169798+00:00,"{'title': 'None', 'author': 'Scooby-Doo', 'tex...","{'quote_react': '𝄞', 'note': None}","{'text_tok': 'Scoobydoobydoo !', 'nb_char': No...",False
9,2024-11-19 22:06:24.169798+00:00,2024-11-19 22:06:24.169798+00:00,"{'title': 'None', 'author': 'Scooby-Doo', 'tex...","{'quote_react': '𝄞', 'note': None}","{'text_tok': 'Scoobydoobydoo !', 'nb_char': No...",False
